<a href="https://colab.research.google.com/github/Tiabet/BaekJoon/blob/main/KoreanSentenceEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BLEURT, BERTSCORE, ROUGE-1 구현

In [ ]:
pip install git+https://github.com/google-research/bleurt.git

In [ ]:
pip install evaluate

In [68]:
pip install rouge

In [79]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [80]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
import evaluate

evaluate.list_evaluation_modules()

사용가능한 평가지표에 bleurt, rouge-1, bertscore 가 모두 있는 것으로 확인된다.
그런데 현재 bleurt 로딩에 뭔가 오류가 있는 것 같다. 그래서 최대한 유사한 google_bleu를 대신 사용하고자 한다.

#데이터 전처리

In [131]:
from konlpy.tag import Mecab
mecab = Mecab()
predictions = ["안녕하세요", "철수는 집을 좋아한다.", "좋아하는 음식이 뭐에요?"]
references = ['안녕하세요', '철수는 집에 가는 것을 좋아한다.', '선호하는 음식이 있으세요?']
for idx, s in enumerate(references):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        references[idx] = tokenized
for idx, s in enumerate(predictions):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        predictions[idx] = tokenized

for idx,sentence in enumerate(predictions):
  together = " ".join(sentence)
  predictions[idx] = together
for idx,sentence in enumerate(references):
  together = " ".join(sentence)
  references[idx] = together
#rouge.get_scores(predictions, references, avg=True)

In [143]:
# Load the BLEU evaluation metric
google_bleu = evaluate.load("google_bleu")

# Compute the BLEU score
results = google_bleu.compute(predictions=predictions, references=references)

# Print the results
results['google_bleu']

0.3918918918918919

In [146]:
bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

In [147]:
sum(results['f1'])/len(predictions)

0.9781759182612101

확인해보니 evaluate의 rouge는 아직 한국어 패키지를 지원하지 않았다. rouge-1 이라는 개념 자체가 그냥 1-gram 씩, 즉 한 단어씩 비교해가면서 같은지 다른지를 체크하는 것. 때문에 형태소 분석을 진행해줘야 하고, 이를 위해서 Mecab을 많이들 사용하는 것 같다. evaluate의 Rouge는 아직 한국어 모델을 정확하게 지원하고 있는 것 같지 않다. (모든 값이 다 0이 나옴) 그래서 rouge를 다른 패키지에서 불러와서 사용하고자 한다.

In [139]:
from rouge import Rouge

rouge = Rouge()
results = rouge.get_scores(predictions, references, avg=True)
results['rouge-1']['f']

0.7749287700381086